In [18]:
#https://chart-studio.plotly.com/~empet/15765/animating-a-3d-orthonormal-basis-http/#/
#https://community.plotly.com/t/how-to-use-plotly-python-for-complex-3d-animations/32983/6
#Note: change "Windowing mode" to 'defer': Improve loading time - Wait for idle CPU cycles
#to attach out of viewport cell, to stop notepad flickering up and down.
#
import plotly.graph_objs as go
import numpy as np
from numpy import pi, sin, cos, sqrt
import panel as pn

pn.extension("plotly")

def spiral(t, r=1, a=3):
    return r*cos(t), r*sin(t), a*t


def tangent(s, r=1, a=3):
    return np.array([-r*sin(s), r*cos(s), a])/sqrt(r**2+a**2)

def binormal(s, r=1, a=3):
    return np.array([a*r*sin(s), -a*r*cos(s), 1])/sqrt(a**2*r**2+1)

def get_frame_data(s,   r=1, a=3, f=1):
    p = spiral(s)
    u1 = tangent(s)
    u3 = binormal(s)
    u2 = np.cross(u3, u1)
    # for visualization purposes we are multiplying the base unit vectors by a factor f
    u1 = f*u1
    u2 = f*u2
    u3 = f*u3
    q = [p+u1, p+u2, p+u3]
    
    #The orthonormal vectors are defined in a single Scatter3d trace, not in three traces. 
    #We are separating by `None` the coordinates of thwo consecutive vectors.
    
    return go.Scatter3d(x = [p[0], q[0][0], None, p[0], q[1][0], None, p[0], q[2][0]],
                        y = [p[1], q[0][1], None, p[1], q[1][1], None, p[1], q[2][1]],
                        z = [p[2], q[0][2], None, p[2], q[1][2], None, p[2], q[2][2]])
    

n_frames = 38 #number of frames
t = np.linspace(0, 3*pi, n_frames)  #parameter values where the spiral and Frenet frames will be evaluated
xs, ys, zs = spiral(t)

fig = go.Figure(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', line_width=4))  #spiral trace

vec_fac = 1 #scaling factor for the three vectors
fig.add_trace(get_frame_data(0, f=vec_fac)) #add the orthonormal basis at the spiral point corresponding to s=0
fig.data[1].update(mode='lines', line_color='red', line_width=7) # update the basis trace by setting vector color, etc

#update layout attributes:
fig.update_scenes( 
                  #xaxis_visible=False, 
                  #yaxis_visible=False, 
                  #zaxis_visible=False,
    
                  #axis autorange=False, below, ensures that axes range  don't enlarge or  
                  #shrink,  according to frame data  (to see its effect, just comment the next three lines)
                  xaxis_autorange=False,
                  yaxis_autorange=False,
                  zaxis_autorange=False,
                  xaxis_range=[xs.min()-1, xs.max()+1],
                  yaxis_range=[ys.min()-1, ys.max()+1],
                  zaxis_range=[zs.min()-1, zs.max()+1], 
                  camera_eye=dict(x=1.2, y=1.2, z=0.8), # camera position
                  #camera_projection_type='orthographic'    #thedefault projection is perspective
)
fig.update_layout(width=800, height=600);
#fig.show()



frames = []

#for  s in t:
#    frames.append(go.Frame(data=[get_frame_data(s, f=vec_fac)],
#                           traces=[1])) #traces=[1] tells Plotly.js that each frame updates fig.data[1], i.e. the vectors

for  k, s in enumerate(t):
    frames.append(go.Frame(data=[get_frame_data(s, f=vec_fac)],
                           name = f'fr{k}',
                           traces=[1])) 
    
fig.update(frames=frames); 

sliders =  [dict(steps= [dict(method= 'animate',
                                               args= [ [ f'fr{k}'], #this steps refers to the frame of name f'fr{k}
                                               dict(mode= 'immediate',
                                                    frame= dict( duration=100, redraw= True ),
                                                    fromcurrent=True,
                                                    transition=dict( duration= 0))],
                                                label=f"fr{k}") for k, s  in enumerate(t)], #this is the step label on the slider 
                                  minorticklen=0,
                                  x=0,
                                  len=1)]
fig.update_layout(sliders=sliders)

fig.update_layout(updatemenus=[dict(type='buttons', 
                                y=0.2,
                                x=1.05,
                                active=0,
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=1000, #HERE you can change the frame rate
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='afterall')])])]);

#Mode mode = 'next' which is the mode of transition between frames. ‘Next’ automatically interpolates the datapoints, meaning we get a gradual effect. Other common options include 'immediate' and 'afterall'
#no difference between 'immediate' and 'afterall' as regards rotating fig with mouse while animating
#can we pause the animation on mouse down and restart it on ouse up?

#Events
#click_data (dict): Click event data from plotly_click event.

#clickannotation_data (dict): Clickannotation event data from plotly_clickannotation event.

#hover_data (dict): Hover event data from plotly_hover and plotly_unhover events.

#relayout_data (dict): Relayout event data from plotly_relayout event

#restyle_data (dict): Restyle event data from plotly_restyle event

#selected_data (dict): Selected event data from plotly_selected and plotly_deselect events.

#viewport (dict): Current viewport state, i.e. the x- and y-axis limits of the displayed plot. Updated on plotly_relayout, plotly_relayouting and plotly_restyle events.

#viewport_update_policy (str, default = ‘mouseup’): Policy by which the viewport parameter is updated during user interactions

#mouseup: updates are synchronized when mouse button is released after panning

#continuous: updates are synchronized continually while panning

#throttle: updates are synchronized while panning, at intervals determined by the viewport_update_throttle parameter

#viewport_update_throttle (int, default = 200, bounds = (0, None)): Time interval in milliseconds at which viewport updates are synchronized when viewport_update_policy is “throttle”.

def on_click(event):
    if event.name == 'click_data':
        if event.obj is not None:
            print("on_click", event.obj.click_data)
            print("on_click old", event.old)
            print("on_click new", event.new)

#fig#.show()
plotly_pane = pn.pane.Plotly(fig)#,viewport_update_policy='continuous')
plotly_pane.param.watch(on_click, ["click_data"],onlychanged=False)
plotly_pane

Plotly(Figure)